In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [7]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
print(X)
net(X)

tensor([[0.5132, 0.6932, 0.7111, 0.3557, 0.1657, 0.1735, 0.8753, 0.4282, 0.1257,
         0.1748, 0.6017, 0.2492, 0.5752, 0.2416, 0.6447, 0.0706, 0.2384, 0.9750,
         0.2418, 0.8943],
        [0.4446, 0.1785, 0.6322, 0.6493, 0.0826, 0.5309, 0.2089, 0.8012, 0.6679,
         0.2291, 0.6403, 0.6514, 0.9909, 0.8030, 0.6633, 0.0669, 0.0657, 0.2133,
         0.2757, 0.0710]])


tensor([[-0.2317,  0.0562,  0.1273,  0.1162,  0.0912,  0.0611, -0.0127, -0.0210,
         -0.2803, -0.0707],
        [-0.1722, -0.0123,  0.0658,  0.0303,  0.0232, -0.0550, -0.0129, -0.1396,
         -0.1181, -0.0894]], grad_fn=<AddmmBackward>)

In [8]:
n = nn.Linear(20, 256)
n.weight

Parameter containing:
tensor([[-0.1368,  0.1192,  0.1843,  ...,  0.0494,  0.0179, -0.0090],
        [-0.1003,  0.0246, -0.0276,  ...,  0.2115, -0.1758, -0.1411],
        [ 0.1360, -0.1925,  0.1439,  ...,  0.0310, -0.0286, -0.1877],
        ...,
        [ 0.1598,  0.0505,  0.1028,  ..., -0.1554, -0.1681, -0.1419],
        [ 0.1122,  0.0039,  0.1382,  ...,  0.0099, -0.1774,  0.1059],
        [-0.1478,  0.1396, -0.1456,  ..., -0.0331,  0.0556, -0.1629]],
       requires_grad=True)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-1.1964e-03, -1.4190e-02, -1.6924e-02, -2.0765e-01,  8.2195e-05,
         -2.6813e-01, -7.8418e-02,  1.1378e-01, -1.8950e-01,  8.2533e-02],
        [-6.5965e-02,  3.3313e-02, -5.6047e-02, -2.0751e-01,  5.1632e-02,
         -2.9583e-01, -5.7584e-02,  8.9808e-02, -1.9287e-01,  3.0272e-01]],
       grad_fn=<AddmmBackward>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0739,  0.0536,  0.1846,  0.1277,  0.1828, -0.1207, -0.0620,  0.0156,
         -0.0364,  0.0773],
        [-0.0285,  0.0085,  0.1338,  0.1334,  0.2319, -0.1149, -0.1675,  0.0134,
         -0.0556,  0.0846]], grad_fn=<AddmmBackward>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.0692, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

In [12]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2655, grad_fn=<SumBackward0>)

In [20]:
class MySequentialList(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.module_list = []
        for block in args:
            self.module_list.append(block)
    
    def forward(self, X):
        for block in self.module_list:
            X = block(X)
        return X
    
net = MySequentialList(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
for module in net.modules():
    print(module)
net(X)

MySequentialList()


tensor([[ 0.0046, -0.0441, -0.0126, -0.0029, -0.1204,  0.0009, -0.2274, -0.1539,
          0.0723, -0.1390],
        [-0.1043,  0.0567, -0.0502, -0.1006, -0.1364, -0.0640, -0.0518,  0.0574,
          0.0085, -0.2234]], grad_fn=<AddmmBackward>)